In [34]:
import pandas as pd
import numpy as np
import re
import string
import pickle

In [35]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

In [36]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [37]:
with open('../static/model/model.pickle','rb') as f:
    model = pickle.load(f)

In [38]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header = None)
tokens = vocab[0].tolist()

In [39]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [40]:
def preprocessing(text):
    df = pd.DataFrame([text], columns=['tweet'])
    df['tweet'] = df['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))

    df['tweet'] = df['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))

    df['tweet'] = df['tweet'].apply(remove_punctuations)

    df['tweet'] = df['tweet'].str.replace('\d+','',regex=True)

    df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x for x in x.split() if x not in sw))

    df['tweet'] = df['tweet'].apply(lambda x: ' '.join(ps.stem(x) for x in x.split()))
    return df['tweet']

In [41]:
def vectorizer(ds,vocabulary):
    vectorized_list = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_list.append(sentence_lst)
        
        
    vectorized_list_new = np.asarray(vectorized_list,dtype = np.float32)
    
    return vectorized_list_new

In [44]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction==1:
        return 'Negative'
    else:
        return 'Positive'

In [47]:
txt = 'good product.i love it.awsome'
preprocessed_text = preprocessing(txt)
vectorized_text = vectorizer(preprocessed_text,tokens)
prediction = get_prediction(vectorized_text)
prediction

'Positive'